In [ ]:
import csv
import cassandra
import dask
import shutil
import pandas as pd
import numpy as np
import time
import dask.dataframe as da
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider1 = PlainTextAuthProvider(username='orderhub_test', password='2345/*')
primary_kcp_cluster = Cluster(['u060dcxyz'],port = 8142,auth_provider=auth_provider1)  
session1 = primary_kcp_cluster.connect('orders')

# starting file number
files = 1
# Max file number
end_file = 3


try:    
    directory= 'apps/python/deletion'
       
    while(files < end_file):
            
            df_read_deletefiles= da.read_csv(f'{directory}/comparing/mark_deletion_{files}.csv')
            #print(df_read_deletefiles['order_id'])
            df_count = df_read_deletefiles['order_id']                      
            order_id_todelete = ', '.join(f"'{delete_id}'" for delete_id in df_count)
            
            df_query = "Delete from orders.kcp_events where order_id in (%s)"%order_id_todelete
            df_query_result = session1.execute(df_query)
            print("querytime ended: "+str(time.time()))  

       #Removing the files from original path to backup folder
            original = f'{directory}/mark_deletion_{files}.csv'
            # #print("done")
            target = (f'/apps/python/completed/mark_deletion{files}.csv')
            shutil.move(original, target)    
            print('completed file:', files )
            
        # Incrementing the file 
            files+=1                              
            print("time finished:" +str(time.time()))

except Exception as e:
   print("No File found for Deletion")
   files += 1 
